### BERT & SQUAD

In questo modulo andremo a scaricare una versione pretrainata di BERT, e procederemo al fine tuning del modello su SQUAD 1.1.

In [5]:
# Stiamo scaricando la versione LARGE del modello, volendo possiamo fare lo stesso finetuning anche su versioni con meno parametri
import os

if not os.path.exists("./multi_cased_L-12_H-768_A-12.zip") or not os.path.exists("./multi_cased_L-12_H-768_A-12"):
    !wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip && unzip multi_cased_L-12_H-768_A-12.zip

if not os.path.exists("./squad_base"):
    !wget <link to squad_base.zip> && unzip squad_base.zip

In [7]:
!export BERT_BASE_DIR=./multi_cased_L-12_H-768_A-12 && export SQUAD_DIR=../squad-exploration && python run_squad.py \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --do_train=True \
  --train_file=$SQUAD_DIR/train-v1.1.json \
  --do_predict=True \
  --predict_file=$SQUAD_DIR/dev-v1.1.json \
  --train_batch_size=12 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --do_lower_case=False \
  --output_dir=./squad_base/




W0616 18:18:13.002625 140080330979136 module_wrapper.py:139] From run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0616 18:18:13.002719 140080330979136 module_wrapper.py:139] From run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0616 18:18:13.002804 140080330979136 module_wrapper.py:139] From /home/calogero/Documents/DeepLearningItalia/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0616 18:18:13.003462 140080330979136 module_wrapper.py:139] From run_squad.py:1133: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.co

### Inference
Adesso che abbiamo allenato il nostro modello dobbiamo ingegnerizzare le classi del repository di BERT in modo da poterle usare in prediction

In [1]:
from run_squad import *
import run_squad

In [2]:
BERT_BASE_DIR = "./multi_cased_L-12_H-768_A-12"
SQUAD_DIR = "../squad-exploration"

FLAGS = {
    "bert_config_file": BERT_BASE_DIR+"/bert_config.json",
    "output_dir": "./squad_base/",
    "vocab_file": BERT_BASE_DIR+"/vocab.txt",
    "do_lower_case": False,
    "use_tpu": False,
    "tpu_name": None,
    "tpu_zone": None,
    "gcp_project": None,
    "master": None,
    "save_checkpoints_steps": 1000,
    "iterations_per_loop": 1000,
    "num_tpu_cores": 8,
    "do_train": False,
    "train_file": SQUAD_DIR+"/train-v1.1.json",
    "train_batch_size": 12,
    "num_train_epochs": 2.0,
    "warmup_proportion": 0.1,
    "init_checkpoint": None,
    "learning_rate": 3e-5,
    "use_tpu": False,
    "predict_batch_size": 8,
    "max_seq_length": 384,
    "doc_stride": 128,
    "max_query_length": 64,
    "do_predict": True,
    "predict_file": SQUAD_DIR+"/dev-v1.1.json",
    "max_answer_length": 30,
    "n_best_size": 20,
    "version_2_with_negative": False,
    "null_score_diff_threshold": 0.0,
    "verbose_logging": False,
    
    
}

In [3]:
def validate_flags_or_throw(bert_config):
    """Validate the input FLAGS or throw an exception."""
    tokenization.validate_case_matches_checkpoint(FLAGS["do_lower_case"],
                                                FLAGS["init_checkpoint"])

    if not FLAGS["do_train"] and not FLAGS["do_predict"]:
        raise ValueError("At least one of `do_train` or `do_predict` must be True.")

    if FLAGS["do_train"]:
        if not FLAGS["train_file"]:
            raise ValueError(
          "If `do_train` is True, then `train_file` must be specified.")
    if FLAGS["do_predict"]:
        if not FLAGS["predict_file"]:
            raise ValueError(
          "If `do_predict` is True, then `predict_file` must be specified.")

    if FLAGS["max_seq_length"] > bert_config.max_position_embeddings:
        raise ValueError(
        "Cannot use sequence length %d because the BERT model "
        "was only trained up to sequence length %d" %
        (FLAGS["max_seq_length"], bert_config.max_position_embeddings))

    if FLAGS["max_seq_length"] <= FLAGS["max_query_length"] + 3:
        raise ValueError(
        "The max_seq_length (%d) must be greater than max_query_length "
        "(%d) + 3" % (FLAGS["max_seq_length"], FLAGS["max_query_length"]))

In [4]:
def get_final_text(pred_text, orig_text, do_lower_case):
    """Project the tokenized prediction back to the original text."""

    # When we created the data, we kept track of the alignment between original
    # (whitespace tokenized) tokens and our WordPiece tokenized tokens. So
    # now `orig_text` contains the span of our original text corresponding to the
    # span that we predicted.
    #
    # However, `orig_text` may contain extra characters that we don't want in
    # our prediction.
    #
    # For example, let's say:
    #   pred_text = steve smith
    #   orig_text = Steve Smith's
    #
    # We don't want to return `orig_text` because it contains the extra "'s".
    #
    # We don't want to return `pred_text` because it's already been normalized
    # (the SQuAD eval script also does punctuation stripping/lower casing but
    # our tokenizer does additional normalization like stripping accent
    # characters).
    #
    # What we really want to return is "Steve Smith".
    #
    # Therefore, we have to apply a semi-complicated alignment heruistic between
    # `pred_text` and `orig_text` to get a character-to-charcter alignment. This
    # can fail in certain cases in which case we just return `orig_text`.

    def _strip_spaces(text):
        ns_chars = []
        ns_to_s_map = collections.OrderedDict()
        for (i, c) in enumerate(text):
            if c == " ":
                continue
            ns_to_s_map[len(ns_chars)] = i
            ns_chars.append(c)
        ns_text = "".join(ns_chars)
        return (ns_text, ns_to_s_map)

    # We first tokenize `orig_text`, strip whitespace from the result
    # and `pred_text`, and check if they are the same length. If they are
    # NOT the same length, the heuristic has failed. If they are the same
    # length, we assume the characters are one-to-one aligned.
    tokenizer = tokenization.BasicTokenizer(do_lower_case=do_lower_case)

    tok_text = " ".join(tokenizer.tokenize(orig_text))

    start_position = tok_text.find(pred_text)
    if start_position == -1:
        if FLAGS["verbose_logging"]:
            tf.logging.info(
          "Unable to find text: '%s' in '%s'" % (pred_text, orig_text))
        return orig_text
    end_position = start_position + len(pred_text) - 1

    (orig_ns_text, orig_ns_to_s_map) = _strip_spaces(orig_text)
    (tok_ns_text, tok_ns_to_s_map) = _strip_spaces(tok_text)

    if len(orig_ns_text) != len(tok_ns_text):
        if FLAGS["verbose_logging"]:
            tf.logging.info("Length not equal after stripping spaces: '%s' vs '%s'",
                      orig_ns_text, tok_ns_text)
        return orig_text

    # We then project the characters in `pred_text` back to `orig_text` using
    # the character-to-character alignment.
    tok_s_to_ns_map = {}
    for (i, tok_index) in six.iteritems(tok_ns_to_s_map):
        tok_s_to_ns_map[tok_index] = i

    orig_start_position = None
    if start_position in tok_s_to_ns_map:
        ns_start_position = tok_s_to_ns_map[start_position]
        if ns_start_position in orig_ns_to_s_map:
            orig_start_position = orig_ns_to_s_map[ns_start_position]

    if orig_start_position is None:
        if FLAGS["verbose_logging"]:
            tf.logging.info("Couldn't map start position")
        return orig_text

    orig_end_position = None
    if end_position in tok_s_to_ns_map:
        ns_end_position = tok_s_to_ns_map[end_position]
        if ns_end_position in orig_ns_to_s_map:
            orig_end_position = orig_ns_to_s_map[ns_end_position]

    if orig_end_position is None:
        if FLAGS["verbose_logging"]:
            tf.logging.info("Couldn't map end position")
        return orig_text

    output_text = orig_text[orig_start_position:(orig_end_position + 1)]
    return output_text

In [5]:
def write_predictions(all_examples, all_features, all_results, n_best_size,
                      max_answer_length, do_lower_case, output_prediction_file,
                      output_nbest_file, output_null_log_odds_file):
    """Write final predictions to the json file and log-odds of null if needed."""
    tf.logging.info("Writing predictions to: %s" % (output_prediction_file))
    tf.logging.info("Writing nbest to: %s" % (output_nbest_file))

    example_index_to_features = collections.defaultdict(list)
    for feature in all_features:
        example_index_to_features[feature.example_index].append(feature)

    unique_id_to_result = {}
    for result in all_results:
        unique_id_to_result[result.unique_id] = result

    _PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
      "PrelimPrediction",
      ["feature_index", "start_index", "end_index", "start_logit", "end_logit"])

    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    scores_diff_json = collections.OrderedDict()

    for (example_index, example) in enumerate(all_examples):
        features = example_index_to_features[example_index]

        prelim_predictions = []
        # keep track of the minimum score of null start+end of position 0
        score_null = 1000000  # large and positive
        min_null_feature_index = 0  # the paragraph slice with min mull score
        null_start_logit = 0  # the start logit at the slice with min null score
        null_end_logit = 0  # the end logit at the slice with min null score
        for (feature_index, feature) in enumerate(features):
            result = unique_id_to_result[feature.unique_id]
            start_indexes = run_squad._get_best_indexes(result.start_logits, n_best_size)
            end_indexes = run_squad._get_best_indexes(result.end_logits, n_best_size)
            # if we could have irrelevant answers, get the min score of irrelevant
            if FLAGS["version_2_with_negative"]:
                feature_null_score = result.start_logits[0] + result.end_logits[0]
                if feature_null_score < score_null:
                    score_null = feature_null_score
                    min_null_feature_index = feature_index
                    null_start_logit = result.start_logits[0]
                    null_end_logit = result.end_logits[0]
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # We could hypothetically create invalid predictions, e.g., predict
                    # that the start of the span is in the question. We throw out all
                    # invalid predictions.
                    if start_index >= len(feature.tokens):
                        continue
                    if end_index >= len(feature.tokens):
                        continue
                    if start_index not in feature.token_to_orig_map:
                        continue
                    if end_index not in feature.token_to_orig_map:
                        continue
                    if not feature.token_is_max_context.get(start_index, False):
                        continue
                    if end_index < start_index:
                        continue
                    length = end_index - start_index + 1
                    if length > max_answer_length:
                        continue
                    prelim_predictions.append(
                      _PrelimPrediction(
                          feature_index=feature_index,
                          start_index=start_index,
                          end_index=end_index,
                          start_logit=result.start_logits[start_index],
                          end_logit=result.end_logits[end_index]))

        if FLAGS["version_2_with_negative"]:
            prelim_predictions.append(
              _PrelimPrediction(
                  feature_index=min_null_feature_index,
                  start_index=0,
                  end_index=0,
                  start_logit=null_start_logit,
                  end_logit=null_end_logit))
            
        prelim_predictions = sorted(
            prelim_predictions,
            key=lambda x: (x.start_logit + x.end_logit),
            reverse=True)

        _NbestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
            "NbestPrediction", ["text", "start_logit", "end_logit"])

        seen_predictions = {}
        nbest = []
        for pred in prelim_predictions:
            if len(nbest) >= n_best_size:
                break
            feature = features[pred.feature_index]
            if pred.start_index > 0:  # this is a non-null prediction
                tok_tokens = feature.tokens[pred.start_index:(pred.end_index + 1)]
                orig_doc_start = feature.token_to_orig_map[pred.start_index]
                orig_doc_end = feature.token_to_orig_map[pred.end_index]
                orig_tokens = example.doc_tokens[orig_doc_start:(orig_doc_end + 1)]
                tok_text = " ".join(tok_tokens)

                # De-tokenize WordPieces that have been split off.
                tok_text = tok_text.replace(" ##", "")
                tok_text = tok_text.replace("##", "")

                # Clean whitespace
                tok_text = tok_text.strip()
                tok_text = " ".join(tok_text.split())
                orig_text = " ".join(orig_tokens)

                final_text = get_final_text(tok_text, orig_text, do_lower_case)
                if final_text in seen_predictions:
                    continue

                seen_predictions[final_text] = True
            else:
                final_text = ""
                seen_predictions[final_text] = True

            nbest.append(
              _NbestPrediction(
                  text=final_text,
                  start_logit=pred.start_logit,
                  end_logit=pred.end_logit))

        # if we didn't inlude the empty option in the n-best, inlcude it
        if FLAGS["version_2_with_negative"]:
            if "" not in seen_predictions:
                nbest.append(
                    _NbestPrediction(
                        text="", start_logit=null_start_logit,
                        end_logit=null_end_logit))
        # In very rare edge cases we could have no valid predictions. So we
        # just create a nonce prediction in this case to avoid failure.
        if not nbest:
            nbest.append(
              _NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0))

        assert len(nbest) >= 1

        total_scores = []
        best_non_null_entry = None
        for entry in nbest:
            total_scores.append(entry.start_logit + entry.end_logit)
            if not best_non_null_entry:
                if entry.text:
                    best_non_null_entry = entry

        probs = run_squad._compute_softmax(total_scores)

        nbest_json = []
        for (i, entry) in enumerate(nbest):
            output = collections.OrderedDict()
            output["text"] = entry.text
            output["probability"] = probs[i]
            output["start_logit"] = entry.start_logit
            output["end_logit"] = entry.end_logit
            nbest_json.append(output)

        assert len(nbest_json) >= 1

        if not FLAGS["version_2_with_negative"]:
            all_predictions[example.qas_id] = nbest_json[0]["text"]
        else:
          # predict "" iff the null score - the score of best non-null > threshold
            score_diff = score_null - best_non_null_entry.start_logit - (
              best_non_null_entry.end_logit)
            scores_diff_json[example.qas_id] = score_diff
            if score_diff > FLAGS["null_score_diff_threshold"]:
                all_predictions[example.qas_id] = ""
            else:
                all_predictions[example.qas_id] = best_non_null_entry.text

        all_nbest_json[example.qas_id] = nbest_json

    with tf.gfile.GFile(output_prediction_file, "w") as writer:
        writer.write(json.dumps(all_predictions, indent=4) + "\n")

    with tf.gfile.GFile(output_nbest_file, "w") as writer:
        writer.write(json.dumps(all_nbest_json, indent=4) + "\n")

    if FLAGS["version_2_with_negative"]:
        with tf.gfile.GFile(output_null_log_odds_file, "w") as writer:
            writer.write(json.dumps(scores_diff_json, indent=4) + "\n")

In [6]:
tf.logging.set_verbosity(tf.logging.INFO)

bert_config = modeling.BertConfig.from_json_file(FLAGS["bert_config_file"])

validate_flags_or_throw(bert_config)

tf.gfile.MakeDirs(FLAGS["output_dir"])

tokenizer = tokenization.FullTokenizer(
    vocab_file=FLAGS["vocab_file"], do_lower_case=FLAGS["do_lower_case"])

tpu_cluster_resolver = None
if FLAGS["use_tpu"] and FLAGS["tpu_name"]:
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
        FLAGS["tpu_name"], zone=FLAGS["tpu_zone"], project=FLAGS["gcp_project"])

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster = tpu_cluster_resolver,
    master = FLAGS["master"],
    model_dir = FLAGS["output_dir"],
    save_checkpoints_steps = FLAGS["save_checkpoints_steps"],
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop = FLAGS["iterations_per_loop"],
        num_shards = FLAGS["num_tpu_cores"],
        per_host_input_for_training=is_per_host
    )
)

train_examples = None
num_train_steps = None
num_warmup_steps = None

if FLAGS["do_train"]:
    train_examples = read_squad_examples(
        input_file=FLAGS["train_file"], is_training=True)
    num_train_steps = int(
        len(train_examples) / FLAGS["train_batch_size"] * FLAGS["num_train_epochs"])
    num_warmup_steps = int(num_train_steps * FLAGS["warmup_proportion"])

    # Pre-shuffle the input to avoid having to make a very large shuffle
    # buffer in in the `input_fn`.
    rng = random.Random(12345)
    rng.shuffle(train_examples)

model_fn = model_fn_builder(
    bert_config=bert_config,
    init_checkpoint=FLAGS["init_checkpoint"],
    learning_rate=FLAGS["learning_rate"],
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=FLAGS["use_tpu"],
    use_one_hot_embeddings=FLAGS["use_tpu"])

# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU.
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS["use_tpu"],
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS["train_batch_size"],
    predict_batch_size=FLAGS["predict_batch_size"])

if FLAGS["do_train"]:
    # We write to a temporary file to avoid storing very large constant tensors
    # in memory.
    train_writer = FeatureWriter(
        filename=os.path.join(FLAGS["output_dir"], "train.tf_record"),
        is_training=True)
    convert_examples_to_features(
        examples=train_examples,
        tokenizer=tokenizer,
        max_seq_length=FLAGS["max_seq_length"],
        doc_stride=FLAGS["doc_stride"],
        max_query_length=FLAGS["max_query_length"],
        is_training=True,
        output_fn=train_writer.process_feature)
    train_writer.close()

    tf.logging.info("***** Running training *****")
    tf.logging.info("  Num orig examples = %d", len(train_examples))
    tf.logging.info("  Num split examples = %d", train_writer.num_features)
    tf.logging.info("  Batch size = %d", FLAGS["train_batch_size"])
    tf.logging.info("  Num steps = %d", num_train_steps)
    del train_examples

    train_input_fn = input_fn_builder(
        input_file=train_writer.filename,
        seq_length=FLAGS["max_seq_length"],
        is_training=True,
        drop_remainder=True)
    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

if FLAGS["do_predict"]:
    eval_examples = read_squad_examples(
        input_file=FLAGS["predict_file"], is_training=False)

    eval_writer = FeatureWriter(
        filename=os.path.join(FLAGS["output_dir"], "eval.tf_record"),
        is_training=False)
    eval_features = []

    def append_feature(feature):
        eval_features.append(feature)
        eval_writer.process_feature(feature)

    convert_examples_to_features(
        examples=eval_examples,
        tokenizer=tokenizer,
        max_seq_length=FLAGS["max_seq_length"],
        doc_stride=FLAGS["doc_stride"],
        max_query_length=FLAGS["max_query_length"],
        is_training=False,
        output_fn=append_feature)
    eval_writer.close()

    tf.logging.info("***** Running predictions *****")
    tf.logging.info("  Num orig examples = %d", len(eval_examples))
    tf.logging.info("  Num split examples = %d", len(eval_features))
    tf.logging.info("  Batch size = %d", FLAGS["predict_batch_size"])

    all_results = []

    predict_input_fn = input_fn_builder(
        input_file=eval_writer.filename,
        seq_length=FLAGS["max_seq_length"],
        is_training=False,
        drop_remainder=False)

    # If running eval on the TPU, you will need to specify the number of
    # steps.
    all_results = []
    for result in estimator.predict(
        predict_input_fn, yield_single_examples=True):
        if len(all_results) % 1000 == 0:
            tf.logging.info("Processing example: %d" % (len(all_results)))
        unique_id = int(result["unique_ids"])
        start_logits = [float(x) for x in result["start_logits"].flat]
        end_logits = [float(x) for x in result["end_logits"].flat]
        all_results.append(
          RawResult(
              unique_id=unique_id,
              start_logits=start_logits,
              end_logits=end_logits))

    output_prediction_file = os.path.join(FLAGS["output_dir"], "predictions.json")
    output_nbest_file = os.path.join(FLAGS["output_dir"], "nbest_predictions.json")
    output_null_log_odds_file = os.path.join(FLAGS["output_dir"], "null_odds.json")

    write_predictions(eval_examples, eval_features, all_results,
                      FLAGS["n_best_size"], FLAGS["max_answer_length"],
                      FLAGS["do_lower_case"], output_prediction_file,
                      output_nbest_file, output_null_log_odds_file)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': './squad_base/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_